In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras import layers, models

In [2]:
dataset, info = tfds.load('plant_village', split='train', with_info=True)

In [3]:
# def preprocess_image(example):
#     image = example['image']
#     # Resize the image to a common size (e.g., 256x256)
#     image = tf.image.resize(image, [256, 256])
#     # Normalize the image
#     image = tf.image.per_image_standardization(image)
#     return image, example['label']

In [4]:
# dataset = dataset.map(preprocess_image)

In [5]:
dataset = dataset.shuffle(buffer_size=info.splits['train'].num_examples)
train_size = int(0.7 * info.splits['train'].num_examples)
val_size = int(0.2 * info.splits['train'].num_examples)
test_size = info.splits['train'].num_examples - train_size - val_size

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)

print("Number of examples in training set:", train_size)
print("Number of examples in validation set:", val_size)
print("Number of examples in test set:", test_size)

Number of examples in training set: 38012
Number of examples in validation set: 10860
Number of examples in test set: 5431


In [6]:
# train_images = np.array([example['image'].numpy() for example in train_dataset])
# train_labels = np.array([example['label'].numpy() for example in train_dataset])
# val_images = np.array([example['image'].numpy() for example in val_dataset])
# val_labels = np.array([example['label'].numpy() for example in val_dataset])
# test_images = np.array([example['image'].numpy() for example in test_dataset])
# test_labels = np.array([example['label'].numpy() for example in test_dataset])

In [22]:
# model = models.Sequential([
#     layers.Conv2D(filters=128, kernel_size=(3, 3),input_shape=(256,256,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)), 
# #     layers.Dropout(0.3),
#     layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)), 
# #     layers.Dropout(0.3),
    
#     layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)), 
# #     layers.Dropout(0.3),
#     layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
# #     layers.Dropout(0.3),
    
#     layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)), 
# #     layers.Dropout(0.3),
#     layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
# #     layers.Dropout(0.2),
    
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
# #     layers.Dropout(0.3),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(38, activation='softmax')
# ])

In [9]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dropout, Input

# Load the model from H5 file
model = load_model('plant_disease_1.h5')

# Create a dictionary to store inputs and outputs of each layer
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Create a new input tensor
new_input = Input(shape=model.input_shape[1:])

# Initialize the previous layer as the new input tensor
x = new_input

# Iterate through the layers of the original model
for layer in model.layers:
    # Check if the layer is not a dropout layer
    if not isinstance(layer, Dropout):
        # Connect the previous layer to the current layer in the new model
        x = layer(x)

# Create the new model
model_without_dropout = Model(new_input, x)

In [10]:
# from tensorflow.keras.models import load_model
# path = "plant_disease_1.h5"
# model = load_model(path)
model_without_dropout.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
model_without_dropout.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        7379

In [12]:
def extract_data_labels(dataset):
    images = []
    labels = []
    for example in dataset:
        images.append(example['image'].numpy())
        labels.append(example['label'].numpy())
    return np.array(images), np.array(labels)

In [20]:
train_images, train_labels = extract_data_labels(test_dataset)

2023-12-25 19:25:35.780484: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 44939 of 54303
2023-12-25 19:25:37.991227: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


In [25]:
model_without_dropout.fit(train_images,train_labels, epochs=5, batch_size=64)

Epoch 1/5
85/85 [==============================] - 810s 10s/step - loss: 0.5698 - accuracy: 0.8415
Epoch 2/5
85/85 [==============================] - 816s 10s/step - loss: 0.3462 - accuracy: 0.8886
Epoch 3/5
85/85 [==============================] - 1488s 18s/step - loss: 0.2234 - accuracy: 0.9273
Epoch 4/5
85/85 [==============================] - 2834s 32s/step - loss: 0.1494 - accuracy: 0.9527
Epoch 5/5
12/85 [===>..........................] - ETA: 13:33 - loss: 0.1623 - accuracy: 0.9479

KeyboardInterrupt: 

In [26]:
model_without_dropout.save("plant_disease_3.h5")